### DateTime Intervals, Aggregates ... 

In [2]:
# import Pandas version > 1.10.  locally: use base(python 3.9.12)
import pandas as pd 
pd.set_option('expand_frame_repr', False)
# pd.set_option('display.max_columns', 0) # Display any number of columns
# pd.set_option('display.max_rows', 0) # Display any number of rows
# set max columns to display
pd.set_option('display.max_columns', 50)

In [3]:
# importing data  "C:\\August2022\\ES 12-22 Globex_2022_09_21.csv"
initial_data = pd.read_csv("C:\\August2022\\ES 12-22 Globex_2022_09_21.csv", parse_dates=['DateTime'])

In [4]:
initial_data.head()

,DateTime,Vol,BidAsk,Bid,Ask,Price
0,2022-09-21 00:00:00:720,1,AtBid,3879.50,3879.75,3879.50
1,2022-09-21 00:00:00:720,1,AtBid,3879.50,3879.75,3879.50
2,2022-09-21 00:00:00:900,1,AtAsk,3879.25,3879.75,3879.75
3,2022-09-21 00:00:00:900,1,AtAsk,3879.25,3879.75,3879.75
4,2022-09-21 00:00:00:900,1,AtAsk,3879.25,3879.75,3879.75


In [ ]:
initial_data.shape # (1742042, 6)

In [12]:
initial_data[' BidAsk'].unique()

array(['AtBid', 'AtAsk', 'BelowBid', 'AboveAsk', 'BetweenBidAsk'],
      dtype=object)

In [8]:
map_BA = {'AtBid': -1, 'BelowBid': -2, 'AtAsk':1, 'AboveAsk': 2, 'BetweenBidAsk': 0}
initial_data[' BidAsk']
initial_data.head(3)

,DateTime,Vol,BidAsk,Bid,Ask,Price
0,2022-09-21 00:00:00:720,1,AtBid,3879.50,3879.75,3879.50
1,2022-09-21 00:00:00:720,1,AtBid,3879.50,3879.75,3879.50
2,2022-09-21 00:00:00:900,1,AtAsk,3879.25,3879.75,3879.75


In [5]:
#  select few columns 
columns_to_select = ['item_name', 'item_code', 'created_at', 'size', 'quantity', 'price', 'store_type']
data = initial_data[columns_to_select]
data.sample(5)   # sample rows

,item_name,item_code,created_at,size,quantity,price,store_type
4778,"Energy saving light bulb, 15W, WKB",110552103,2016-05-08 19:37:44.934,1.0,1,60.0,small_medium_shop
22362,"Men's haircut, barber shop",111211101,2016-05-30 13:35:19.022,1.0,1,180.0,private_service_provider
12607,"Pizza marguerite, at restaurant",111111105,2016-06-30 15:36:16.572,1.0,1,60.0,private_service_provider
21008,"Football (soccer ball), WKB",110931110,2016-02-22 14:21:26.276,1.0,1,199.0,small_medium_shop
29297,"Men's lace-up shoes, WKB-L",110321101,2016-04-11 00:17:49.661,1.0,1,3350.0,small_medium_shop


## Aggregating data based on Intervals  Hour, Month, Offset...    
- Combining data based on different Time Intervals.

In [6]:
# Amount added in each hour 
data.resample('H', on='created_at').price.sum().head(5)

created_at
2015-12-14 18:00:00    5449.90
2015-12-14 19:00:00      15.98
2015-12-14 20:00:00      66.98
2015-12-14 21:00:00       0.00
2015-12-14 22:00:00       0.00
Freq: H, Name: price, dtype: float64

In [7]:
# Changing start time for each hour, by default start time is at 0th minute
# data.resample('H', on='created_at', offset='15Min10s').price.sum().head(5) 
data.resample('H', on='created_at', offset='-15Min0s').price.sum().head(5)  #  offset can be negative  offset='-15Min10s'

created_at
2015-12-14 17:45:00    5449.90
2015-12-14 18:45:00       0.00
2015-12-14 19:45:00      74.76
2015-12-14 20:45:00       8.20
2015-12-14 21:45:00       0.00
Freq: H, Name: price, dtype: float64

In [8]:
# total amount added each week
data.resample('W', on='created_at').price.sum().head(5)

created_at
2015-12-20     43056.38
2015-12-27     67338.51
2016-01-03     44434.59
2016-01-10     18222.36
2016-01-17    190838.54
Freq: W-SUN, Name: price, dtype: float64

In [9]:
# By default, week starts with Sunday. Let's change it to start with Monday
data.resample('W-MON', on='created_at').price.sum().head(5)

created_at
2015-12-14     5532.86
2015-12-21    38507.62
2015-12-28    66863.29
2016-01-04    53924.10
2016-01-11    12608.69
Freq: W-MON, Name: price, dtype: float64

In [10]:
data.resample('M', on='created_at').price.sum()

created_at
2015-12-31    1.538769e+05
2016-01-31    4.297143e+05
2016-02-29    9.352684e+05
2016-03-31    7.425185e+06
2016-04-30    1.384351e+07
2016-05-31    1.253785e+07
2016-06-30    7.264586e+06
2016-07-31    6.739416e+06
2016-08-31    2.721504e+06
Freq: M, Name: price, dtype: float64

In [11]:
# by default month labels are assigned to the last day of month, lets change it to start of the day using "MS"
data.resample('MS', on='created_at', label='left').price.sum()

created_at
2015-12-01    1.538769e+05
2016-01-01    4.297143e+05
2016-02-01    9.352684e+05
2016-03-01    7.425185e+06
2016-04-01    1.384351e+07
2016-05-01    1.253785e+07
2016-06-01    7.264586e+06
2016-07-01    6.739416e+06
2016-08-01    2.721504e+06
Freq: MS, Name: price, dtype: float64

In [13]:
data.resample('H', on='created_at').agg({'price':'sum', 'quantity':'sum','item_code':'nunique'}).head(5)

,price,quantity,item_code
created_at,,,
2015-12-14 18:00:00,5449.90,3,3
2015-12-14 19:00:00,15.98,2,2
2015-12-14 20:00:00,66.98,7,4
2015-12-14 21:00:00,0.00,0,0
2015-12-14 22:00:00,0.00,0,0


In [18]:
data.groupby([pd.Grouper(key='created_at', freq='W'), 'store_type']).price.sum().head(10)

created_at  store_type                
2015-12-20  other                         34300.00
            public_semi_public_service      833.90
            small_medium_shop              1206.04
2015-12-27  small_medium_shop               923.09
            specialized_shop              65164.00
2016-01-03  small_medium_shop               355.10
            specialized_shop              41922.00
2016-01-10  other                          8300.00
            small_medium_shop               412.55
            specialized_shop               3979.00
Name: price, dtype: float64

In [17]:
data.groupby([pd.Grouper(key='created_at', freq='H'), 'store_type']).price.sum().head(10)

created_at           store_type                
2015-12-14 19:00:00  small_medium_shop                10.09
2015-12-14 20:00:00  small_medium_shop                48.58
2015-12-15 16:00:00  small_medium_shop               114.80
2015-12-15 19:00:00  small_medium_shop                62.72
2015-12-16 00:00:00  public_semi_public_service      833.90
2015-12-16 13:00:00  small_medium_shop                32.90
2015-12-16 14:00:00  small_medium_shop                37.90
2015-12-16 20:00:00  other                          7500.00
2015-12-16 21:00:00  other                         26800.00
2015-12-17 18:00:00  small_medium_shop               308.95
Name: price, dtype: float64

In [19]:
data.groupby([pd.Grouper(key='created_at', freq='M'), 'item_name']).price.sum()

created_at  item_name                                   
2015-12-31  Bar soap, solid, SB                                33.17
            Beer, domestic brand, single bottle, WKB           29.79
            Black tea, BL                                      12.00
            Black tea, in bags, WKB                            60.99
            Bread, white, sliced, WKB                          85.45
                                                              ...   
2016-08-31  Wheat flour, not self-rising, BL                  150.38
            White sugar, WKB                                  266.47
            Women's haircut, basic hairdresser               7730.00
            Wrist-watch, men's, CITIZEN Eco-Drive BM6060    52205.00
            Yoghurt, plain, WKB                               150.96
Name: price, Length: 1061, dtype: float64

In [ ]:
data.columns    #  (['item_name', 'item_code', 'created_at', 'size', 'quantity', 'price','store_type'] 

In [26]:
data.groupby([pd.Grouper(key='created_at', freq='M'), 'store_type'])\
    .agg(unique_items_n =('item_name', 'unique'),
         unique_items_list = ('item_code', 'unique'),
         total_quantity=('quantity','sum'),
         total_amount=('price','sum')).head(5)

unique_items_n                                  unique_items_list  total_quantity  total_amount
created_at store_type                                                                                                                                                    
2015-12-31 other                       [One-bedroom apartment, 40-60 m2, Two-bedroom ...               [1104111110, 1104111113, 1104111102]               6      34300.00
           public_semi_public_service                             [Electricity: 600 kWh]                                        [110451101]               1        833.90
           small_medium_shop           [Wheat flour, not self-rising, BL, Jam, strawb...  [110111202, 110118201, 111212111, 110121105, 1...              88       2484.23
           specialized_shop                  [Scooter, 50cc, WKB, Summer tire, R15, WKB]                             [110712104, 110723103]              20     107086.00
2016-01-31 market                      [Charcoal, sack, Tinned pineapple, whole slice...                             [110453101, 110116201]              12        473.75